## This jupyter notebook compares the sage version of macaulay.py from autoguess to the without sage version

## The configuration of input and output file

# Below is the sage version

In [5]:
from sage.all import *
from sage.rings.polynomial.multi_polynomial_sequence import PolynomialSequence
import time
import sys
from datetime import datetime
from argparse import ArgumentParser, RawTextHelpFormatter

class Macaulay:
    """
    Given a set of Boolean polynomials and a positive integer D, as well as a monomial ordering, 
    this class performs two main tasks:
    1- Constructing the Macaulay matrix of degree D
    2- Computing the reduced row echelon form of the derived Macaulay matrix
    """

    count = 0

    def __init__(self, inputfile, outputfile, D=3, term_ordering='deglex'):
        Macaulay.count += 1
        self.inputfile = inputfile
        self.outputfile = outputfile
        self.D = D
        self.term_ordering = term_ordering
        self.algebrize_input_polynomials()

    def algebrize_input_polynomials(self):
        """
        Converts the given polynomials in string format to a list of boolean polynomials of type PolynomialSequence
        """
        try:
            with open(self.inputfile, 'r') as equations_file:
                string_polynomials = equations_file.read().splitlines()
        except IOError:
            print('%s is not accessible!' % self.inputfile)
            sys.exit()
        symbolic_polynomials = list(map(symbolic_expression, string_polynomials))
        symbolic_variables = sorted(set(flatten([list(eq.variables()) for eq in symbolic_polynomials])))
        print("Sage variable order:", symbolic_variables)
        self.PolyRing = BooleanPolynomialRing(len(symbolic_variables), names=symbolic_variables, order=self.term_ordering)
        self.polynomial_sequence = PolynomialSequence([self.PolyRing(eq) for eq in symbolic_polynomials])

    def build_macaulay_polynomials(self):
        """
        Constructs the Macaulay polynomials with degree at most D corresponding to the given boolean polynomial sequence
        """
        self.macaulay_polynomials = []
        nvars = self.PolyRing.n_variables()
        degree_spectrum = sorted(set([f.degree() for f in self.polynomial_sequence]))
        print('Number of algebraic equations: %d' % len(self.polynomial_sequence))
        print('Number of algebraic variables: %d' % nvars)
        print('Number of algebraic monomials: %d' % self.polynomial_sequence.nmonomials())
        print('Spectrum of degrees: %s' % degree_spectrum)
        multiplied_monomials = {}
        for d in degree_spectrum:
            if d < self.D:
                maximal_exponents = IntegerVectors(self.D - d, nvars, max_part=1)
                leading_terms = [self.PolyRing({tuple(exponent): 1}) for exponent in maximal_exponents]
                monomials = list(set(flatten([list(lead_term.lead_divisors()) for lead_term in leading_terms])))
                multiplied_monomials[d] = monomials
            else:
                multiplied_monomials[d] = [1]
        for f in self.polynomial_sequence:
            self.macaulay_polynomials.extend([m * f for m in multiplied_monomials[f.degree()]])
        self.macaulay_polynomials = PolynomialSequence(self.macaulay_polynomials)

    def build_macaulay_matrix(self):
        """
        Generates the Macaulay matrix
        """
        if self.polynomial_sequence.maximal_degree() > 1:
            minimum_degree = min([f.degree() for f in self.polynomial_sequence])
            if self.D < minimum_degree:
                self.D = minimum_degree
            self.build_macaulay_polynomials()
            self.macaulay_matrix, self.macaulay_vars = self.macaulay_polynomials.coefficients_monomials()
        else:
            self.macaulay_matrix, self.macaulay_vars = self.polynomial_sequence.coefficients_monomials()
        print('Macaulay matrix was generated in %s' % str.lower(self.macaulay_matrix.parent().__repr__()))

    def gaussian_elimination(self):
        """
        Applies Gaussian elimination to compute the row reduced echelon form of the derived Macaulay matrix
        """
        print('Gaussian elimination was started - %s' % datetime.now())
        start_time = time.time()
        self.macaulay_matrix.echelonize()
        elapsed_time = time.time() - start_time

        self.dependent_vars = [
            self.macaulay_vars[i]
            for i in self.macaulay_matrix.pivots()
            if self.macaulay_vars[i] not in [0, 1]
        ]

        self.free_vars = [
            self.macaulay_vars[i]
            for i in self.macaulay_matrix.nonpivots()
            if self.macaulay_vars[i] not in [0, 1]
        ]

        print('#Dependent variables: %d' % len(self.dependent_vars))
        print('#Free variables: %d' % len(self.free_vars))
        print('Gaussian elimination was finished after %0.4f seconds' % elapsed_time)

    def write_result(self):
        """
        Writes the derived polynomials into the output file
        """
        nrows = self.macaulay_matrix.nrows()
        print('Writing the results into the %s - %s' % (self.outputfile, datetime.now()))
        starting_time = time.time()
        with open(self.outputfile, 'w') as outputfile:
            for i in range(nrows):
                output = " + ".join(
                    str(self.macaulay_vars[j])
                    for j in self.macaulay_matrix.nonzero_positions_in_row(i)
                )
                if output:
                    outputfile.write(output + "\n")
        elapsed_time = time.time() - starting_time
        print('Result was written into %s after %0.02f seconds' % (self.outputfile, elapsed_time))


print("SAGE VERSION")
print("="*60)

# Configuration
inputfile = 'eqn.txt'
outputfile = 'output_sage.txt'
D = 3

# Execution
total_start = time.time()

macaulay = Macaulay(inputfile=inputfile, outputfile=outputfile, D=D)
macaulay.build_macaulay_matrix()
macaulay.gaussian_elimination()
macaulay.write_result()

total_time = time.time() - total_start
print(f"TOTAL TIME: {total_time:.4f} seconds")
print("="*60)

SAGE VERSION
Sage variable order: [X1, X3, X2, X4]
Number of algebraic equations: 7
Number of algebraic variables: 4
Number of algebraic monomials: 11
Spectrum of degrees: [2]
Macaulay matrix was generated in full matrixspace of 35 by 15 sparse matrices over finite field of size 2
Gaussian elimination was started - 2025-12-31 15:55:04.928481
#Dependent variables: 13
#Free variables: 1
Gaussian elimination was finished after 0.0001 seconds
Writing the results into the output_sage.txt - 2025-12-31 15:55:04.928695
Result was written into output_sage.txt after 0.00 seconds
TOTAL TIME: 0.0075 seconds


# Below is the without sage version

In [3]:
import sys
sys.path.append("/Users/hiren/sourcecodes/pull-req/autoguess-without-sage/core")
from gf2poly import parse_gf2poly, GF2Poly, rref_gf2_u64, validate_packed_gf2_u64

## Galois version

In [4]:
import time
import sys
from datetime import datetime
from itertools import combinations
import numpy as np
import galois


def poly_degree(poly: GF2Poly) -> int:
    """
    Return the total degree of a GF2Poly:
      max number of variables in any monomial.
    """
    # handle the zero polynomial
    if not poly.monomials:
        return 0
    return max(len(mono) for mono in poly.monomials)

class Macaulay:
    count = 0
    
    def __init__(self, inputfile, outputfile, D=2, term_ordering='deglex'):
        Macaulay.count += 1
        self.inputfile = inputfile
        self.outputfile = outputfile
        self.D = D
        self.term_ordering = term_ordering
        self.algebraize_input_polynomials()
        
    def algebraize_input_polynomials(self):
        try:
            with open(self.inputfile, 'r') as eqf:
                # Skip empty lines and comments
                lines = [L.strip() for L in eqf
                         if L.strip() and not L.strip().startswith('#')]
        except IOError:
            print(f'{self.inputfile} is not accessible!')
            sys.exit(1)
        
        # 1) Parse each algebraic relation into GF2Poly
        self.polynomials = [parse_gf2poly(line) for line in lines]  # list of GF2Poly

        # 2) Collect all variable names from monomials
        vars_set = set()
        for poly in self.polynomials:
            for mono in poly.monomials:    # mono is a frozenset of var-names
                vars_set.update(mono)
        self.variable_names = sorted(vars_set)  # e.g. ['x0','x1','x2',…]
    
    def build_macaulay_polynomials(self):
        """
        Constructs the Macaulay polynomials (all in GF2Poly form)
        of total degree ≤ D, given the input GF2Polys in self.polynomials.
        """

        # 1) Prepare
        self.macaulay_polynomials = []
        nvars = len(self.variable_names)

        # 2) Compute degrees of each input GF2Poly via our poly_degree helper
        degrees = [poly_degree(f) for f in self.polynomials]
        degree_spectrum = sorted(set(degrees))

        unique_monos = set()
        for poly in self.polynomials:
            unique_monos |= poly.monomials   # union in all monomials
    
        print('Number of algebraic equations: %d' % len(self.polynomials))
        print('Number of algebraic variables: %d' % nvars)
        print('Number of algebraic monomials: %d' % len(unique_monos))
        print('Spectrum of degrees: %s' % degree_spectrum)


        # 3) Pre-compute the monomial multipliers for each degree d
        #    so that d + deg(multiplier) ≤ D
        multipliers = {}
        for d in degree_spectrum:
            r = self.D - d
            if r < 0:
                # if an equation already exceeds D, just multiply by 1
                multipliers[d] = [GF2Poly([frozenset()])]
                continue

            mons = []
            # generate all monomials of degree k = 0..r
            for k in range(r + 1):
                for idxs in combinations(range(nvars), k):
                    # build the frozenset of names for this monomial
                    mono_vars = frozenset(self.variable_names[i] for i in idxs)
                    mons.append(GF2Poly([mono_vars]))
            multipliers[d] = mons
       
        # 4) Multiply each input poly by its allowed multipliers,
        #    collecting only the non-zero products
        count = 0
        for f_poly, d in zip(self.polynomials, degrees):
            for m in multipliers[d]:
                prod = m * f_poly          # GF2Poly __mul__
                if prod:                   # skip the zero polynomial
                    self.macaulay_polynomials.append(prod)
                    count += 1
                    
        
    def build_macaulay_matrix(self):
        """
        Generates the Macaulay matrix
        """
        # 1) Decide if we need Macaulay lifting (nonlinear case)
        degrees = [poly_degree(f) for f in self.polynomials]
        

        if max(degrees) > 1:
            min_deg = min(degrees)
            if self.D < min_deg:
                self.D = min_deg
            self.build_macaulay_polynomials()
        else:
            self.macaulay_polynomials = list(self.polynomials)

        # 2) Collect & sort all monomials (as exponent‐tuples) according to term ordering
        var_names = self.variable_names
        exps = set()
        for poly in self.macaulay_polynomials:
            for mono in poly.monomials:
                # mono is frozenset of var‐names → build an exp tuple
                exps.add(tuple(1 if v in mono else 0 for v in var_names))

        # Apply the chosen term ordering
        if self.term_ordering == 'degrevlex':
            # Degree reverse lexicographic: by total degree (descending), then reverse lex (descending)
            self.mons = sorted(exps, key=lambda e: (sum(e), tuple(reversed(e))), reverse=True)
        else:  # 'deglex' or default
            # Degree lexicographic: by total degree (descending), then lex (descending)
            self.mons = sorted(exps, key=lambda e: (sum(e), e), reverse=True)

        col_index = {exp: i for i, exp in enumerate(self.mons)}


        # 3) Build the 0/1 matrix
        n_rows = len(self.macaulay_polynomials)
        n_cols = len(self.mons)
        A = np.zeros((n_rows, n_cols), dtype=np.uint8)

        for i, poly in enumerate(self.macaulay_polynomials):
            for mono in poly.monomials:
                exp = tuple(1 if v in mono else 0 for v in var_names)
                A[i, col_index[exp]] = 1

        self.macaulay_matrix = A
        print(f"Macaulay matrix was generated in full matrixspace of {n_rows} by {n_cols} sparse matrices over finite field of size 2")

    def gaussian_elimination(self):
        """
        Applies Gaussian elimination to compute the row reduced echelon form of the derived Macaulay matrix
        """
        print('Gaussian elimination started - %s' % datetime.now())
        t0 = time.time()
        
        GF2 = galois.GF(2)
        A = GF2(self.macaulay_matrix)
        
        self.R = A.row_reduce()
        R_np = np.array(self.R, dtype=np.uint8)
        
        pivots = [int(np.nonzero(R_np[i])[0][0]) for i in range(R_np.shape[0]) if R_np[i].any()]
        
        def is_constant(mono):
            return all(e == 0 for e in mono)
        
        self.dependent = [m for j, m in enumerate(self.mons) if j in pivots and not is_constant(m)]
        self.free = [m for j, m in enumerate(self.mons) if j not in pivots and not is_constant(m)]
        
        elapsed = time.time() - t0
        print(f"#Dependent variables: {len(self.dependent)}")
        print(f"#Free variables: {len(self.free)}")
        print(f"Gaussian elimination was finished after {elapsed:.4f} seconds")
    
    def write_result(self):
        """
        Writes the derived polynomials into the output file
        """
        print('Writing the results into the %s - %s' % (self.outputfile, datetime.now()))
        t0 = time.time()
        try:
            with open(self.outputfile, 'w') as outf:
                # R should be your reduced-row-echelon form (NumPy array or list of lists)
                rows = np.array(self.R, dtype=np.uint8)

                for row in rows:
                    terms = []
                    for j, bit in enumerate(row):
                        if bit:
                            mono = self.mons[j]             # e.g. (0,1,0,1,0,...)
                            if sum(mono) == 0:
                                # constant term
                                terms.append('1')
                            else:
                                # select the variable names for exponent=1
                                var_list = [
                                    self.variable_names[i]
                                    for i, e in enumerate(mono) if e
                                ]
                                terms.append('*'.join(var_list))

                    if terms:
                        equation = ' + '.join(terms)
                        outf.write(equation + '\n')

        except IOError:
            print(f"Error: cannot write to {self.outputfile}")
            sys.exit(1)

        elapsed = time.time() - t0
        print(f"Result was written into {self.outputfile} after {elapsed:.2f} seconds")

print("GALOIS VERSION")
print("="*60)

# Configuration
inputfile = 'eqn.txt'
outputfile = 'output_galois.txt'
D = 3

# Execution
total_start = time.time()

macaulay = Macaulay(inputfile=inputfile, outputfile=outputfile, D=D)
macaulay.build_macaulay_matrix()
macaulay.gaussian_elimination()
macaulay.write_result()

total_time = time.time() - total_start
print(f"TOTAL TIME: {total_time:.4f} seconds")
print("="*60)

GALOIS VERSION
Number of algebraic equations: 7
Number of algebraic variables: 4
Number of algebraic monomials: 11
Spectrum of degrees: [2]
Macaulay matrix was generated in full matrixspace of 35 by 15 sparse matrices over finite field of size 2
Gaussian elimination started - 2025-12-31 15:35:01.892580
#Dependent variables: 13
#Free variables: 1
Gaussian elimination was finished after 0.3304 seconds
Writing the results into the output_galois.txt - 2025-12-31 15:35:02.223131
Result was written into output_galois.txt after 0.00 seconds
TOTAL TIME: 0.3320 seconds


## GF2Poly Version

In [5]:
import time
import sys
from datetime import datetime
from itertools import combinations
import numpy as np
import re

def first_seen_var_names(lines):
    seen = set()
    ordered = []
    for line in lines:
        for name in re.findall(r"[A-Za-z_]\w*", line):
            if name not in seen:
                seen.add(name)
                ordered.append(name)
    return ordered

def poly_degree(poly: GF2Poly) -> int:
    """
    Return the total degree of a GF2Poly:
      max number of variables in any monomial.
    """
    # handle the zero polynomial
    if not poly.monomials:
        return 0
    return max(len(mono) for mono in poly.monomials)


def _pack_bits_to_uint64(mat: np.ndarray) -> tuple[np.ndarray, int]:
    """
    Pack a binary matrix into uint64 words (little-endian bit order).

    Returns (packed_matrix, original_ncols).
    """
    rows, cols = mat.shape
    nwords = (cols + 63) // 64
    padded_cols = nwords * 64
    if padded_cols != cols:
        pad = np.zeros((rows, padded_cols - cols), dtype=np.uint8)
        mat = np.concatenate([mat, pad], axis=1)
    packed = np.packbits(mat, axis=1, bitorder="little")
    packed = np.ascontiguousarray(packed)
    return packed.view(np.uint64), cols


def _pivot_columns_from_packed(mat: np.ndarray, ncols: int) -> list[int]:
    """
    Extract pivot column indices from a packed RREF matrix.
    """
    pivots: list[int] = []
    for row in mat:
        for w, word in enumerate(row):
            word_int = int(word)
            if word_int:
                lsb = word_int & -word_int
                bit = lsb.bit_length() - 1
                col = w * 64 + bit
                if col < ncols:
                    pivots.append(col)
                break
    return pivots


class MacaulayGF2Poly:
    count = 0

    def __init__(self, inputfile, outputfile, D=2, term_ordering='deglex'):
        MacaulayGF2Poly.count += 1
        self.inputfile = inputfile
        self.outputfile = outputfile
        self.D = D
        self.term_ordering = term_ordering
        self.algebraize_input_polynomials()

    def algebraize_input_polynomials(self):
        try:
            with open(self.inputfile, 'r') as eqf:
                # Skip empty lines and comments
                lines = [L.strip() for L in eqf
                         if L.strip() and not L.strip().startswith('#')]
        except IOError:
            print(f'{self.inputfile} is not accessible!')
            sys.exit(1)
        
        # 1) Parse each algebraic relation into GF2Poly
        self.polynomials = [parse_gf2poly(line) for line in lines]  # list of GF2Poly

        # 2) Collect all variable names from monomials
        vars_set = set()
        for poly in self.polynomials:
            for mono in poly.monomials:    # mono is a frozenset of var-names
                vars_set.update(mono)
        self.variable_names = first_seen_var_names(lines)
        print("gf2poly variable order:", self.variable_names)

    def build_macaulay_polynomials(self):
        """
        Constructs the Macaulay polynomials (all in GF2Poly form)
        of total degree ≤ D, given the input GF2Polys in self.polynomials.
        """

        # 1) Prepare
        self.macaulay_polynomials = []
        nvars = len(self.variable_names)

        # 2) Compute degrees of each input GF2Poly via our poly_degree helper
        degrees = [poly_degree(f) for f in self.polynomials]
        degree_spectrum = sorted(set(degrees))

        unique_monos = set()
        for poly in self.polynomials:
            unique_monos |= poly.monomials   # union in all monomials
    
        print('Number of algebraic equations: %d' % len(self.polynomials))
        print('Number of algebraic variables: %d' % nvars)
        print('Number of algebraic monomials: %d' % len(unique_monos))
        print('Spectrum of degrees: %s' % degree_spectrum)


        # 3) Pre-compute the monomial multipliers for each degree d
        #    so that d + deg(multiplier) ≤ D
        multipliers = {}
        for d in degree_spectrum:
            r = self.D - d
            if r < 0:
                # if an equation already exceeds D, just multiply by 1
                multipliers[d] = [GF2Poly([frozenset()])]
                continue

            mons = []
            # generate all monomials of degree k = 0..r
            for k in range(r + 1):
                for idxs in combinations(range(nvars), k):
                    # build the frozenset of names for this monomial
                    mono_vars = frozenset(self.variable_names[i] for i in idxs)
                    mons.append(GF2Poly([mono_vars]))
            multipliers[d] = mons
       
        # 4) Multiply each input poly by its allowed multipliers,
        #    collecting only the non-zero products
        count = 0
        for f_poly, d in zip(self.polynomials, degrees):
            for m in multipliers[d]:
                prod = m * f_poly          # GF2Poly __mul__
                if prod:                   # skip the zero polynomial
                    self.macaulay_polynomials.append(prod)
                    count += 1
                    

    def build_macaulay_matrix(self):
        """
        Generates the Macaulay matrix
        """
        # 1) Decide if we need Macaulay lifting (nonlinear case)
        degrees = [poly_degree(f) for f in self.polynomials]
        

        if max(degrees) > 1:
            min_deg = min(degrees)
            if self.D < min_deg:
                self.D = min_deg
            self.build_macaulay_polynomials()
        else:
            self.macaulay_polynomials = list(self.polynomials)

        # 2) Collect & sort all monomials (as exponent‐tuples) according to term ordering
        var_names = self.variable_names
        exps = set()
        for poly in self.macaulay_polynomials:
            for mono in poly.monomials:
                # mono is frozenset of var‐names → build an exp tuple
                exps.add(tuple(1 if v in mono else 0 for v in var_names))

        # Apply the chosen term ordering
        if self.term_ordering == 'degrevlex':
            # Degree reverse lexicographic: by total degree (descending), then reverse lex (descending)
            self.mons = sorted(exps, key=lambda e: (sum(e), tuple(reversed(e))), reverse=True)
        else:  # 'deglex' or default
            # Degree lexicographic: by total degree (descending), then lex (descending)
            self.mons = sorted(exps, key=lambda e: (sum(e), e), reverse=True)

        col_index = {exp: i for i, exp in enumerate(self.mons)}


        # 3) Build the 0/1 matrix
        n_rows = len(self.macaulay_polynomials)
        n_cols = len(self.mons)
        A = np.zeros((n_rows, n_cols), dtype=np.uint8)

        for i, poly in enumerate(self.macaulay_polynomials):
            for mono in poly.monomials:
                exp = tuple(1 if v in mono else 0 for v in var_names)
                A[i, col_index[exp]] = 1

        self.macaulay_matrix = A
        print(f"Macaulay matrix was generated in full matrixspace of {n_rows} by {n_cols} sparse matrices over finite field of size 2")

    def gaussian_elimination(self):
        """
        Applies Gaussian elimination to compute the row reduced echelon form of the derived Macaulay matrix
        """
        print('Gaussian elimination started - %s' % datetime.now())
        packed, ncols = _pack_bits_to_uint64(self.macaulay_matrix)
        validate_packed_gf2_u64(packed, ncols)
        # Warm up JIT so timing reflects steady-state performance
        rref_gf2_u64(packed.copy())

        t0 = time.time()
        packed, ncols = _pack_bits_to_uint64(self.macaulay_matrix)
        validate_packed_gf2_u64(packed, ncols)
        rref_gf2_u64(packed)

        pivots = _pivot_columns_from_packed(packed, ncols)

        # Unpack for downstream use (e.g., write_result)
        unpacked = np.unpackbits(packed.view(np.uint8), axis=1, bitorder="little")
        self.R = unpacked[:, :ncols].astype(np.uint8, copy=False)
        
        def is_constant(mono):
            return all(e == 0 for e in mono)

        # Exclude constant monomial from free/dependent count
        self.dependent = [m for j, m in enumerate(self.mons) if j in pivots and not is_constant(m)]
        self.free = [m for j, m in enumerate(self.mons) if j not in pivots and not is_constant(m)]

        print(f"#Dependent variables: {len(self.dependent)}")
        print(f"#Free variables: {len(self.free)}")


        elapsed = time.time() - t0
        print(f"Gaussian elimination was finished after {elapsed:.4f} seconds")

    def write_result(self):
        """
        Writes the derived polynomials into the output file
        """
        print('Writing the results into the %s - %s' % (self.outputfile, datetime.now()))
        t0 = time.time()
        try:
            with open(self.outputfile, 'w') as outf:
                # R should be your reduced-row-echelon form (NumPy array or list of lists)
                rows = np.array(self.R, dtype=np.uint8)

                for row in rows:
                    terms = []
                    for j, bit in enumerate(row):
                        if bit:
                            mono = self.mons[j]             # e.g. (0,1,0,1,0,...)
                            if sum(mono) == 0:
                                # constant term
                                terms.append('1')
                            else:
                                # select the variable names for exponent=1
                                var_list = [
                                    self.variable_names[i]
                                    for i, e in enumerate(mono) if e
                                ]
                                terms.append('*'.join(var_list))

                    if terms:
                        equation = ' + '.join(terms)
                        outf.write(equation + '\n')

        except IOError:
            print(f"Error: cannot write to {self.outputfile}")
            sys.exit(1)

        elapsed = time.time() - t0
        print(f"Result was written into {self.outputfile} after {elapsed:.2f} seconds")


print("NUMBA JIT VERSION")
print("="*60)

# Configuration
inputfile = 'eqn.txt'
outputfile = 'output_gf2poly.txt'
D = 3

# Execution
total_start = time.time()

macaulay = MacaulayGF2Poly(inputfile=inputfile, outputfile=outputfile, D=D)
macaulay.build_macaulay_matrix()
macaulay.gaussian_elimination()
macaulay.write_result()

total_time = time.time() - total_start
print(f"TOTAL TIME: {total_time:.4f} seconds")
print("="*60)

NUMBA JIT VERSION
gf2poly variable order: ['X1', 'X3', 'X2', 'X4']
Number of algebraic equations: 7
Number of algebraic variables: 4
Number of algebraic monomials: 11
Spectrum of degrees: [2]
Macaulay matrix was generated in full matrixspace of 35 by 15 sparse matrices over finite field of size 2
Gaussian elimination started - 2025-12-31 16:12:58.855423
#Dependent variables: 13
#Free variables: 1
Gaussian elimination was finished after 0.0023 seconds
Writing the results into the output_gf2poly.txt - 2025-12-31 16:12:58.862269
Result was written into output_gf2poly.txt after 0.00 seconds
TOTAL TIME: 0.0135 seconds
